# 🔌 微网数字孪生系统 - Microgrid Digital Twin System

## 完整功能演示

本notebook展示了一个完整的微网数字孪生系统，包括：

- 🏗️ **微网核心模拟** - 光伏、风电、储能、负荷等组件模型
- 🔮 **预测系统** - 功率、电价、负荷预测
- 🤖 **强化学习能量管理** - 自适应策略生成
- 📊 **策略评估** - 多维度性能评估
- 💬 **自然语言交互** - 文本控制界面
- 🎮 **3D可视化** - 交互式界面

---

## 1. 环境配置

In [ ]:
# 检查运行环境
import sys
import os

# 判断是否在Colab中运行
IN_COLAB = 'google.colab' in sys.modules
print(f"运行环境: {'Google Colab' if IN_COLAB else '本地环境'}")
print(f"Python版本: {sys.version}")

# 确保numpy可用
try:
    import numpy as np
    print(f"NumPy版本: {np.__version__}")
except ImportError:
    !pip install numpy
    import numpy as np

# 如果在Colab中，克隆仓库
if IN_COLAB:
    # 添加当前目录到路径
    if '/content' not in sys.path:
        sys.path.insert(0, '/content')
    print("Colab环境配置完成")

In [ ]:
# 添加模块路径
import sys
import os

# 获取当前工作目录
current_dir = os.getcwd()
if current_dir not in sys.path:
    sys.path.insert(0, current_dir)

print(f"工作目录: {current_dir}")

## 2. 导入系统模块

In [ ]:
# 导入微网数字孪生系统模块
from microgrid_digital_twin.core import MicrogridDigitalTwin
from microgrid_digital_twin.prediction import (
    PowerPredictor, PricePredictor, LoadPredictor, IntegratedForecaster
)
from microgrid_digital_twin.rl_agent import (
    EnergyManagementAgent, RuleBasedAgent, AdaptiveEnergyManager
)
from microgrid_digital_twin.evaluation import StrategyEvaluator, RealTimeMonitor
from microgrid_digital_twin.nlp_interface import NLPInterface
from microgrid_digital_twin.visualization import Visualization3D, generate_3d_visualization_html

import numpy as np
from datetime import datetime, timedelta
import time

print("✅ 所有模块导入成功！")

## 3. 初始化微网数字孪生系统

In [ ]:
# 创建微网数字孪生系统实例
config = {
    'solar': {
        'capacity_kw': 100.0,
        'efficiency': 0.18,
        'panel_area': 500.0
    },
    'wind': {
        'capacity_kw': 50.0,
        'cut_in_speed': 3.0,
        'rated_speed': 12.0
    },
    'battery': {
        'capacity_kwh': 200.0,
        'max_charge_rate': 50.0,
        'max_discharge_rate': 50.0,
        'soc': 0.5
    },
    'diesel': {
        'capacity_kw': 100.0
    },
    'load': {
        'base_load': 80.0,
        'peak_load': 150.0
    },
    'grid': {
        'max_import': 100.0,
        'max_export': 50.0
    }
}

# 创建数字孪生系统
digital_twin = MicrogridDigitalTwin(config)

print("🏗️ 微网数字孪生系统初始化完成")
print(f"   - 光伏容量: {config['solar']['capacity_kw']} kW")
print(f"   - 风电容量: {config['wind']['capacity_kw']} kW")
print(f"   - 储能容量: {config['battery']['capacity_kwh']} kWh")
print(f"   - 峰值负荷: {config['load']['peak_load']} kW")

## 4. 查看系统当前状态

In [ ]:
# 获取系统当前状态
state = digital_twin.get_state()

print("📊 【微网系统当前状态】")
print(f"⏰ 时间: {state['timestamp']}")
print("")
print("🔋 发电设备:")
print(f"   ☀️ 光伏发电: {state['components']['solar']['current_power']:.2f} kW")
print(f"   💨 风力发电: {state['components']['wind']['current_power']:.2f} kW")
print("")
print("🔌 储能系统:")
print(f"   电池容量: {state['components']['battery']['capacity']:.0f} kWh")
print(f"   当前SOC: {state['components']['battery']['soc']*100:.1f}%")
print(f"   健康度: {state['components']['battery']['health']*100:.1f}%")
print("")
print("📈 负荷与电网:")
print(f"   当前负荷: {state['components']['load']['current']:.2f} kW")
print(f"   电网状态: {'已连接' if state['components']['grid']['connected'] else '离网'}")
print("")
print("🌤️ 天气条件:")
print(f"   太阳辐照度: {state['weather']['irradiance']:.0f} W/m²")
print(f"   风速: {state['weather']['wind_speed']:.1f} m/s")
print(f"   温度: {state['weather']['temperature']:.1f} °C")
print("")
print("💰 电价信息:")
print(f"   购电价格: ¥{state['price']['buy_price']:.2f}/kWh")
print(f"   售电价格: ¥{state['price']['sell_price']:.2f}/kWh")
print(f"   时段类型: {state['price']['period']}")

## 5. 预测系统演示

In [ ]:
# 创建综合预测系统
forecaster = IntegratedForecaster(prediction_horizon=60)  # 预测未来60分钟

# 获取当前时间
now = datetime.now()
hour = now.hour
minute = now.minute

print(f"🔮 【预测系统 - 未来60分钟预测】")
print(f"⏰ 当前时间: {now.strftime('%H:%M')}")
print("")

# 获取预测结果
forecasts = forecaster.forecast_all(hour, minute, state['weather'])

print("☀️ 光伏发电预测:")
print(f"   平均功率: {np.mean(forecasts['solar']['mean']):.1f} kW")
print(f"   最大功率: {np.max(forecasts['solar']['mean']):.1f} kW")
print(f"   预测区间: {np.min(forecasts['solar']['lower_95']):.1f} - {np.max(forecasts['solar']['upper_95']):.1f} kW")
print("")
print("💨 风力发电预测:")
print(f"   平均功率: {np.mean(forecasts['wind']['mean']):.1f} kW")
print(f"   不确定度: ±{np.mean(forecasts['wind']['std']):.1f} kW")
print("")
print("📊 负荷预测:")
print(f"   平均负荷: {np.mean(forecasts['load']['mean']):.1f} kW")
print(f"   峰值负荷: {np.max(forecasts['load']['mean']):.1f} kW")
print("")
print("💰 电价预测:")
print(f"   平均电价: ¥{np.mean(forecasts['price']['mean']):.2f}/kWh")

## 6. 强化学习能量管理系统

In [ ]:
# 创建能量管理智能体
rl_agent = EnergyManagementAgent(
    state_dim=10,
    action_dim=2,
    hidden_dims=[128, 64],
    learning_rate=0.001,
    gamma=0.99,
    epsilon=1.0,
    epsilon_decay=0.995
)

# 创建规则based智能体作为对比
rule_agent = RuleBasedAgent()

# 创建自适应能量管理器
adaptive_manager = AdaptiveEnergyManager()

print("🤖 能量管理系统初始化完成")
print(f"   RL智能体状态维度: 10")
print(f"   RL智能体动作维度: 2 (电池控制 + 柴油机开关)")
print(f"   初始探索率: {rl_agent.epsilon:.2f}")

In [ ]:
# 训练强化学习智能体
print("🎮 开始训练强化学习智能体...")
print("")

# 重置系统
digital_twin.reset()

# 训练参数
n_episodes = 5  # 训练轮数
steps_per_episode = 60  # 每轮步数（模拟1小时）

episode_rewards = []
episode_costs = []

for episode in range(n_episodes):
    digital_twin.reset()
    total_reward = 0
    
    for step in range(steps_per_episode):
        # 获取当前观测
        obs = digital_twin.get_observation()
        state_dict = digital_twin.get_state()
        
        # 选择动作
        action = adaptive_manager.select_action(obs, state_dict, training=True)
        
        # 执行动作
        next_state = digital_twin.step(action)
        next_obs = digital_twin.get_observation()
        
        # 计算奖励
        reward = rl_agent.calculate_reward(state_dict, action, next_state)
        total_reward += reward
        
        # 训练
        done = (step == steps_per_episode - 1)
        adaptive_manager.train(obs, action, reward, next_obs, done)
    
    episode_rewards.append(total_reward)
    episode_costs.append(digital_twin.total_cost)
    
    print(f"  Episode {episode+1}/{n_episodes}: 总奖励={total_reward:.2f}, 成本=¥{digital_twin.total_cost:.2f}, ε={rl_agent.epsilon:.3f}")

print("")
print("✅ 训练完成！")
print(f"   平均奖励: {np.mean(episode_rewards):.2f}")
print(f"   平均成本: ¥{np.mean(episode_costs):.2f}")

## 7. 运行模拟并评估策略

In [ ]:
# 创建策略评估器
evaluator = StrategyEvaluator()
monitor = RealTimeMonitor()

# 运行完整模拟（模拟2小时）
print("🔄 开始模拟运行...")
print("")

digital_twin.reset()
simulation_steps = 120  # 2小时

for step in range(simulation_steps):
    # 获取观测
    obs = digital_twin.get_observation()
    state_dict = digital_twin.get_state()
    
    # 使用训练好的策略选择动作
    action = adaptive_manager.select_action(obs, state_dict, training=False)
    
    # 执行动作
    result = digital_twin.step(action)
    
    # 更新监控器
    monitor.update(result)
    
    # 每30步输出一次状态
    if (step + 1) % 30 == 0:
        print(f"  步骤 {step+1}/{simulation_steps}: 光伏={result['solar_power']:.1f}kW, "
              f"风电={result['wind_power']:.1f}kW, 负荷={result['load_power']:.1f}kW, "
              f"SOC={result['battery_soc']*100:.1f}%, 成本=¥{result['total_cost']:.2f}")

print("")
print("✅ 模拟完成！")

In [ ]:
# 评估运行结果
metrics = evaluator.evaluate_episode(digital_twin.history)
report = evaluator.generate_report(metrics, "2小时模拟")

print(report)

## 8. 自然语言交互界面

In [ ]:
# 创建自然语言接口
nlp_interface = NLPInterface(
    digital_twin=digital_twin,
    agent=adaptive_manager.rl_agent,
    evaluator=evaluator,
    forecaster=forecaster
)

print("💬 自然语言交互系统已启动")
print("")
print("您可以使用以下命令与系统交互：")
print("  - '查看系统状态'")
print("  - '电池电量'")
print("  - '当前电价'")
print("  - '天气情况'")
print("  - '帮助'")
print("")

In [ ]:
# 演示自然语言交互
demo_queries = [
    "查看系统状态",
    "电池电量怎么样",
    "当前电价多少",
    "可再生能源利用率"
]

print("=" * 60)
print("💬 自然语言交互演示")
print("=" * 60)

for query in demo_queries:
    print("")
    print(f"👤 用户: {query}")
    print("-" * 40)
    response = nlp_interface.process(query)
    print(f"🤖 系统: \n{response}")
    print("")

In [ ]:
# 交互式对话（可选）
def chat(message):
    """与系统对话"""
    response = nlp_interface.process(message)
    return response

# 示例：获取帮助
print(chat("帮助"))

## 9. 3D可视化界面 🎮

In [ ]:
# 创建3D可视化
visualization = Visualization3D(digital_twin)

# 生成HTML
html_content = visualization.generate()

print(f"✅ 3D可视化界面已生成")
print(f"   HTML大小: {len(html_content) / 1024:.1f} KB")

In [ ]:
# 保存HTML文件
html_file = "microgrid_3d_visualization.html"
visualization.save_html(html_file)

print(f"\n📁 3D可视化界面已保存到: {html_file}")
print("\n您可以通过以下方式查看：")
print("  1. 直接在浏览器中打开此HTML文件")
print("  2. 使用下方的显示代码在notebook中预览")

In [ ]:
# 在notebook中显示3D可视化
from IPython.display import HTML, IFrame, display

# 方法1: 直接显示HTML（适用于Colab和Jupyter）
try:
    display(HTML(html_content))
except Exception as e:
    print(f"直接显示失败: {e}")
    print("请尝试下载HTML文件并在浏览器中打开")

In [ ]:
# 如果在Colab中，提供下载选项
if IN_COLAB:
    from google.colab import files
    files.download(html_file)
    print("\n📥 文件下载已开始，请在浏览器中打开下载的HTML文件查看3D可视化界面")

## 10. 策略对比分析

In [ ]:
# 对比RL策略和规则策略
print("📊 策略对比分析")
print("="*60)

strategies_results = {}

# 1. RL策略
print("\n运行RL策略...")
digital_twin.reset()
for _ in range(60):
    obs = digital_twin.get_observation()
    state_dict = digital_twin.get_state()
    action = rl_agent.select_action(obs, training=False)
    digital_twin.step(action)
strategies_results['RL策略'] = dict(digital_twin.history)
rl_cost = digital_twin.total_cost

# 2. 规则策略
print("运行规则策略...")
digital_twin.reset()
for _ in range(60):
    state_dict = digital_twin.get_state()
    # 构造规则策略需要的状态字典
    rule_state = {
        'battery_soc': digital_twin.battery.soc,
        'electricity_price': state_dict['price']['buy_price'],
        'solar_power': state_dict['components']['solar']['current_power'],
        'wind_power': state_dict['components']['wind']['current_power'],
        'load_power': state_dict['components']['load']['current']
    }
    action = rule_agent.select_action(rule_state)
    digital_twin.step(action)
strategies_results['规则策略'] = dict(digital_twin.history)
rule_cost = digital_twin.total_cost

# 3. 无策略（随机）
print("运行随机策略...")
digital_twin.reset()
for _ in range(60):
    action = {
        'battery_action': np.random.uniform(-0.5, 0.5),
        'diesel_on': False
    }
    digital_twin.step(action)
strategies_results['随机策略'] = dict(digital_twin.history)
random_cost = digital_twin.total_cost

print("\n=" * 60)
print("策略成本对比:")
print(f"  🤖 RL策略:   ¥{rl_cost:.2f}")
print(f"  📋 规则策略: ¥{rule_cost:.2f}")
print(f"  🎲 随机策略: ¥{random_cost:.2f}")
print("")

# 计算节省
if random_cost > 0:
    rl_saving = (random_cost - rl_cost) / random_cost * 100
    rule_saving = (random_cost - rule_cost) / random_cost * 100
    print(f"相比随机策略:")
    print(f"  RL策略节省: {rl_saving:.1f}%")
    print(f"  规则策略节省: {rule_saving:.1f}%")

In [ ]:
# 详细策略对比
comparison = evaluator.compare_strategies(strategies_results)

print("\n📈 详细对比分析:")
print("=" * 60)

for strategy_name, metrics in comparison.items():
    if strategy_name == 'rankings':
        continue
    print(f"\n【{strategy_name}】")
    print(f"  净成本: ¥{metrics['cost_metrics']['net_cost']:.2f}")
    print(f"  可再生能源比例: {metrics['energy_metrics']['renewable_ratio']:.1f}%")
    print(f"  电网依赖度: {metrics['grid_metrics']['grid_dependency']:.1f}%")
    print(f"  CO2排放: {metrics['environmental_metrics']['co2_emissions']:.2f} kg")

## 11. 系统状态监控

In [ ]:
# 实时监控统计
stats = monitor.get_statistics()

print("📊 实时监控统计（最近60分钟）")
print("=" * 60)

for component, data in stats.items():
    print(f"\n{component.upper()}:")
    print(f"  当前值: {data['current']:.2f} kW")
    print(f"  平均值: {data['mean']:.2f} kW")
    print(f"  最大值: {data['max']:.2f} kW")
    print(f"  最小值: {data['min']:.2f} kW")
    print(f"  标准差: {data['std']:.2f} kW")

In [ ]:
# 查看告警
alerts = monitor.get_recent_alerts()

print("\n⚠️ 最近告警:")
print("=" * 60)

if alerts:
    for alert in alerts:
        level_icon = "⚠️" if alert['level'] == 'warning' else "ℹ️"
        print(f"{level_icon} [{alert['time'].strftime('%H:%M:%S')}] {alert['message']}")
else:
    print("✅ 暂无告警")

## 12. 自适应管理器状态

In [ ]:
# 查看自适应能量管理器状态
manager_status = adaptive_manager.get_status()

print("🧠 自适应能量管理器状态")
print("=" * 60)
print(f"  运行模式: {manager_status['mode']}")
print(f"  RL置信度: {manager_status['rl_confidence']*100:.1f}%")
print(f"  探索率(ε): {manager_status['epsilon']:.4f}")
print(f"  训练步数: {manager_status['training_steps']}")
print(f"  经验池大小: {manager_status['buffer_size']}")
print(f"  近期性能: {manager_status['recent_performance']:.2f}")

## 📝 总结

本notebook展示了完整的微网数字孪生系统功能：

1. **核心模拟系统** - 完整的微网组件模型
2. **预测系统** - 多目标时序预测与不确定性估计
3. **强化学习能量管理** - 自适应决策优化
4. **策略评估** - 多维度性能评估与对比
5. **自然语言交互** - 用户友好的对话接口
6. **3D可视化** - 沉浸式监控界面

### 下一步

- 打开生成的 `microgrid_3d_visualization.html` 文件体验交互式3D界面
- 尝试更多自然语言命令与系统交互
- 调整系统参数观察不同场景下的表现

In [ ]:
print("\n" + "="*60)
print("🎉 微网数字孪生系统演示完成！")
print("="*60)
print("\n感谢使用本系统！")
print("\n如需进一步帮助，请使用自然语言接口输入 '帮助' 查看可用功能。")